In [1]:
import os
import time
import pandas as pd
from datetime import datetime

In [2]:
# 数据路径
data_path = r"F:\collection_spb_info\XJSH\ALL_DATA\ALL_RESULT\ALL_TABLE\52_贵州省_result_20250109.xlsx"

In [3]:
df = pd.read_excel(data_path)

In [4]:
# 提取信息
use_col = ['ydbh','dwjd','dwwd','dwgc','ydlb','yypbh','jcjg','yjz','ph','ylzjhl','qdan','qlin','qjia','qxi','yxlin','sxjia','hxjia','zgong','zshen','zqian','zge','zge2','jxzc1trzd','gzchd_x','gzchd_y','yxtchd','fsxtl','fsxyl','fsxts','fsxtz','my', 'myqt', 'mz', 'mzqt']

In [115]:
# col_dict = {'ydbh':"YDBH",'dwjd':"DWJD",'dwwd':"DWWD",'dwgc':"DWGC",'ydlb','yypbh','jcjg','yjz','ph','ylzjhl','qdan','qlin','qjia','qxi','yxlin','sxjia','hxjia','zgong','zshen','zqian','zge','zge2','jxzc1trzd','gzchd_x','gzchd_y','yxtchd','fsxtl','fsxyl','fsxts','fsxtz'}

In [6]:
dy_df = df[df['xjdm']==520502]

In [7]:
dy_df = dy_df[use_col]

In [8]:
len(dy_df['ydbh'].unique()),len(dy_df['yypbh'].unique()),dy_df.shape[0]

(2739, 3172, 3242)

In [9]:
# 去除yypbh结尾不为10和11的
dy_df['yypbh'] = dy_df['yypbh'].astype(str)
filter_dy_df = dy_df[dy_df['yypbh'].str.endswith('10') | dy_df['yypbh'].str.endswith('11')]
len(filter_dy_df['ydbh'].unique()),len(filter_dy_df['yypbh'].unique()),filter_dy_df.shape[0]

(2739, 2739, 2802)

In [10]:
# 去除yypbh重复的数据，只保留第一个
re_filter_dy_df = filter_dy_df.drop_duplicates(subset=['yypbh'], keep='first')
len(re_filter_dy_df['ydbh'].unique()),len(re_filter_dy_df['yypbh'].unique()),re_filter_dy_df.shape[0]

(2739, 2739, 2739)

In [11]:
# 计算gzchd,(gzchd_x,gzchd_y,两列不为空和不为0的值，如果为空和0，则取0)
def cal_gzchd(row):
    try:
        if float(row['gzchd_x']) != 0:
            return float(row['gzchd_x'])
        elif float(row['gzchd_y']) != 0:
            return float(row['gzchd_y'])
        else:
            return 0
    except:
        return 0

In [12]:
result_col = use_col = ['ydbh','dwjd','dwwd','dwgc','ydlb','yypbh','jcjg','yjz','ph','ylzjhl','qdan','qlin','qjia','qxi','yxlin','sxjia','hxjia','zgong','zshen','zqian','zge','zge2','jxzc1trzd','gzchd_x','gzchd_y','yxtchd','fsxtl','fsxyl','fsxts','fsxtz','my', 'myqt', 'mz', 'mzqt']

In [13]:
result_df = re_filter_dy_df[result_col]

In [14]:
# 获取耕作层厚度
result_df['gzchd'] = result_df.apply(cal_gzchd,axis=1)
# 删除gzchd_x,gzchd_y
result_df = result_df.copy()
result_df.drop(columns=['gzchd_x','gzchd_y'],inplace=True)

In [15]:
mymz_dict = {
    'AS': '风积沙', 'LO': '原生黄土', 'LOP': '黄土状物质（次生黄土）', 'LI': '坡积物',
    'LG': '坡积物', 'MA': '洪积物', 'FL': '冲积物', 'PY': '海积物',
    'AL': '湖积物', 'VA': '河流沉积物', 'CO': '火成碎屑沉积物', 'WE': '冰川沉积物',
    'SA': '有机沉积物', 'CD': '崩积物', 'QR': '红黏土', 'OT': '其他',
    '01': '第四纪松散沉积物', '02': '花岗岩', '03': '流纹岩', '04': '闪长岩',
    '05': '安山岩', '06': '正长岩', '07': '粗面岩', '08': '辉长岩',
    '09': '玄武岩', '10': '橄榄岩', '11': '脉岩', '12': '块集岩',
    '13': '火山角砾岩', '14': '凝灰岩', '15': '角砾岩', '16': '砾岩',
    '17': '砂岩', '18': '页岩', '19': '化学石灰岩', '20': '生物石灰岩',
    '21': '白云岩', '22': '片麻岩', '23': '石英岩', '24': '板岩',
    '25': '结晶片岩', '26': '大理岩', '27': '泥岩', '99': '其他'
}
# 返回muyanmz信息
def get_muyanmz(value):
    # 处理空值和NaN
    if pd.isna(value):
        return ''
    # 转换为字符串
    value = str(value)
    # 考虑多个组合的情况
    if ',' in value:
        value_list = value.split(',')
        result_list = []
        for val in value_list:
            try:
                result_list.append(mymz_dict.get(val, ''))
            except:
                result_list.append(val)
        return ','.join(result_list)
    else:
        try:
            return mymz_dict.get(value, '')
        except:
            return value

In [16]:
# 计算my myqt mz mzqt
result_df['my'] = result_df['my'].apply(get_muyanmz)
result_df['mz'] = result_df['mz'].apply(get_muyanmz)
result_df['myqt'] = result_df['myqt'].apply(get_muyanmz)
result_df['mzqt'] = result_df['mzqt'].apply(get_muyanmz)

In [17]:
# 导出
result_df['ydbh'] = result_df['ydbh'].astype(str)
# 筛选列
filter_result_df = result_df[['ydbh','dwjd','dwwd','ydlb','my','myqt','mz','mzqt']]

In [18]:
# 使用dwjd、dwwd将df转换为shp点文件
import geopandas as gpd
from shapely.geometry import Point

# 创建geometry列
geometry = [Point(xy) for xy in zip(filter_result_df['dwjd'], filter_result_df['dwwd'])]

# 转换为GeoDataFrame
gdf = gpd.GeoDataFrame(filter_result_df, geometry=geometry)

# 设置坐标系统为WGS84
gdf.crs = "EPSG:4326"
out_path =r"C:\Users\Runker\Desktop\shp\qxg_points.shp"
os.makedirs(os.path.dirname(out_path), exist_ok=True)
# 导出为shp文件
gdf.to_file(out_path, encoding='utf-8')
